In [20]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
from animal_Shelter import AnimalShelter

# Connect to MongoDB
shelter = AnimalShelter("aacuser", "MySecurePassword123")

# Create the Dash app
app = JupyterDash('CS340ProjectTwo')

# Define layout
app.layout = html.Div([
    # Logo at the top, centered
    html.Div([
        html.Img(src='files/photo.png', style={'height': '160px'}),
    ], style={'textAlign': 'center'}),

    html.H1("Kain Mason - SNHU CS-340 Dashboard", style={'textAlign': 'center'}),
    html.Hr(),

    # Rescue Type Filter Section
    html.Div([
        html.Label("Select Rescue Type:"),
        dcc.RadioItems(
            id='rescue-type-filter',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'block'}
        )
    ], style={'width': '25%', 'float': 'left'}),

    # DataTable Section
    html.Div([
        dash_table.DataTable(
            id='datatable-interactivity',
            columns=[],
            data=[],
            filter_action='native',
            sort_action='native',
            row_selectable='single',
            selected_rows=[],
            page_size=10,
            style_table={'overflowX': 'auto'},
            style_cell={
                'minWidth': '100px',
                'width': '150px',
                'maxWidth': '250px',
                'overflow': 'hidden',
                'textOverflow': 'ellipsis'
            }
        )
    ], style={'width': '70%', 'float': 'right'}),

    html.Div(style={'clear': 'both'}),
    html.Hr(),

    # Map Output
    html.Div(id='map-output'),
    html.Hr(),

    # Chart Output
    html.Div(id='chart-output'),

    html.Footer("MongoDB Dashboard - Kain Mason", style={'textAlign': 'center', 'fontWeight': 'bold'})
])

# Callback to update table based on filter
@app.callback(
    Output('datatable-interactivity', 'data'),
    Output('datatable-interactivity', 'columns'),
    Input('rescue-type-filter', 'value')
)
def update_table(rescue_type):
    if rescue_type == 'Water Rescue':
        breeds = ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]
        query = {"breed": {"$in": breeds}, "age_upon_outcome_in_weeks": {"$lte": 156}}
    elif rescue_type == 'Mountain or Wilderness Rescue':
        breeds = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]
        query = {"breed": {"$in": breeds}, "age_upon_outcome_in_weeks": {"$lte": 156}}
    elif rescue_type == 'Disaster or Individual Tracking':
        breeds = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
        query = {"breed": {"$in": breeds}, "age_upon_outcome_in_weeks": {"$lte": 156}}
    else:
        query = {}

    results = shelter.read(query)
    df = pd.DataFrame.from_records(results)
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

    # Add rescue_type column
    if rescue_type != 'Reset':
        df['rescue_type'] = rescue_type
    else:
        df['rescue_type'] = df['breed'].apply(lambda b: (
            'Water Rescue' if b in ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"] else
            'Mountain or Wilderness Rescue' if b in ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"] else
            'Disaster or Individual Tracking' if b in ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"] else
            'Other'
        ))

    columns = [{"name": i, "id": i} for i in df.columns]
    return df.to_dict('records'), columns

# Callback for map and outcome chart
@app.callback(
    Output('map-output', "children"),
    Output('chart-output', "children"),
    [Input('datatable-interactivity', "derived_virtual_data"),
     Input('datatable-interactivity', "derived_virtual_selected_rows")]
)
def update_widgets(rows, selected_rows):
    if not selected_rows or not rows:
        map_out = html.P("Select a row to view location on map.")
        chart_out = html.P("Waiting for data...")
        return map_out, chart_out

    row = rows[selected_rows[0]]
    try:
        lat = float(row['location_lat'])
        lon = float(row['location_long'])
        map_out = dl.Map(center=[lat, lon], zoom=12, children=[
            dl.TileLayer(),
            dl.Marker(position=[lat, lon])
        ], style={'width': '100%', 'height': '500px'})
    except:
        map_out = html.P("Invalid location data.")

    df = pd.DataFrame(rows)
    if 'outcome_type' in df.columns:
        fig = px.histogram(df, x='outcome_type', title='Outcome Type Distribution')
        chart_out = dcc.Graph(figure=fig)
    else:
        chart_out = html.P("No outcome data available.")

    return map_out, chart_out

# Run the app
app.run_server(port=8057)


Connected successfully to MongoDB.
Dash app running on http://127.0.0.1:8057/
Found 9936 document(s) matching query.
Found 9936 document(s) matching query.
Found 9936 document(s) matching query.
Found 9936 document(s) matching query.
Found 9936 document(s) matching query.
Found 9936 document(s) matching query.
Found 9936 document(s) matching query.
Found 9936 document(s) matching query.
